In [9]:

import sys
import os
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))
from models import resnet_model
from models import efficient_model
from models import mobilenet_model

data_path = os.path.join(os.path.dirname(os.getcwd()), 'data')
sys.path.append(data_path)

# 4. Importer DataLoaders depuis dataloaders.py
from DataLoaders import DataLoader

import tensorflow as tf
import tensorflow.keras.backend as K
import gc

In [2]:
print(f"Keras version: {tf.keras.__version__}")


Keras version: 3.8.0


In [3]:

data_loader = DataLoader(path="C:/Users/nessa/Downloads/sipakmed_new6/")
datasets = data_loader.get_datasets()

train_with_aug = datasets["train_with_augmentation"]
test_set = datasets["test"]

Found 551 images belonging to 3 classes.
Found 551 images belonging to 3 classes.
Found 177 images belonging to 3 classes.


In [4]:
callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
    EarlyStopping(monitor='val_loss', patience=15, verbose=1, restore_best_weights=True),
]

In [6]:
def train_and_evaluate(model_class, model_name):

    print("\n==============================")
    print(f"  Entraînement du modèle : {model_name}")
    print("==============================\n")

    model_obj = model_class(
        input_shape=(224, 224, 3),
        num_classes=3,
        learning_rate=1e-4
    )
    model = model_obj.get_model()

    # ---- PHASE 1 ----
    model.fit(
        train_with_aug,
        epochs=45,
        validation_data=test_set,
        callbacks=callbacks,
        verbose=2
    )

    # ---- FINE TUNING ----
    for layer in model.layers:
        layer.trainable = True

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    model.fit(
        train_with_aug,
        epochs=10,
        validation_data=test_set,
        callbacks=callbacks,
        verbose=2
    )

    # ================ ÉVALUATION ================
    print(f"\nÉvaluation du modèle {model_name} :")
    test_loss, test_accuracy = model.evaluate(test_set, verbose=2)
    print(f"> Accuracy : {test_accuracy * 100:.2f}%")

    # ==== EXTRACTION LABELS ====
    y_true = []
    for i in range(len(test_set)):
        _, label = test_set[i]   # (image, one-hot)
        y_true.append(label[0])

    y_true = np.array(y_true)
    y_true_classes = np.argmax(y_true, axis=1)

    # ==== PRÉDICTIONS ====
    y_pred = model.predict(test_set)
    y_pred_classes = np.argmax(y_pred, axis=1)

    # ==== AUC ====
    auc = roc_auc_score(y_true, y_pred, multi_class='ovr')
    print(f"> AUC : {auc:.4f}")

    # ==== RAPPORT ====
    print("\nRapport de classification :")
    print(classification_report(
        y_true_classes,
        y_pred_classes,
        target_names=data_loader.class_names
    ))

    # ==== MATRICE ====
    print("\nMatrice de confusion :")
    print(confusion_matrix(y_true_classes, y_pred_classes))

    return {
        "model": model_name,
        "accuracy": test_accuracy,
        "auc": auc
    }


In [7]:
results = []

results.append(train_and_evaluate(ResNet50_Model, "ResNet50"))
K.clear_session()
gc.collect()

results.append(train_and_evaluate(EfficientNetB0_Model, "EfficientNetB0"))
K.clear_session()
gc.collect()

results.append(train_and_evaluate(MobileNetV2_Model, "MobileNetV2"))
K.clear_session()
gc.collect()



  Entraînement du modèle : ResNet50



c:\Users\nessa\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/45
18/18 - 299s - 17s/step - accuracy: 0.6425 - loss: 5.3451 - val_accuracy: 0.6441 - val_loss: 5.2366 - learning_rate: 1.0000e-04
Epoch 2/45
18/18 - 224s - 12s/step - accuracy: 0.8675 - loss: 4.7669 - val_accuracy: 0.8362 - val_loss: 4.8356 - learning_rate: 1.0000e-04
Epoch 3/45
18/18 - 224s - 12s/step - accuracy: 0.8984 - loss: 4.5888 - val_accuracy: 0.7401 - val_loss: 5.2134 - learning_rate: 1.0000e-04
Epoch 4/45
18/18 - 236s - 13s/step - accuracy: 0.9456 - loss: 4.4025 - val_accuracy: 0.8362 - val_loss: 4.7450 - learning_rate: 1.0000e-04
Epoch 5/45
18/18 - 216s - 12s/step - accuracy: 0.9710 - loss: 4.2440 - val_accuracy: 0.8475 - val_loss: 4.5975 - learning_rate: 1.0000e-04
Epoch 6/45
18/18 - 229s - 13s/step - accuracy: 0.9655 - loss: 4.1345 - val_accuracy: 0.8249 - val_loss: 4.7803 - learning_rate: 1.0000e-04
Epoch 7/45
18/18 - 223s - 12s/step - accuracy: 0.9782 - loss: 4.0145 - val_accuracy: 0.9040 - val_loss: 4.2408 - learning_rate: 1.0000e-04
Epoch 8/45
18/18 - 224s - 1

c:\Users\nessa\OneDrive\Bureau\tp\Apprentisaage-1\Mobile.py:17: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  backbone = MobileNetV2(


Epoch 1/45
18/18 - 136s - 8s/step - accuracy: 0.5318 - loss: 5.2721 - val_accuracy: 0.2316 - val_loss: 5.8315 - learning_rate: 1.0000e-04
Epoch 2/45
18/18 - 85s - 5s/step - accuracy: 0.7060 - loss: 4.8038 - val_accuracy: 0.2486 - val_loss: 6.3243 - learning_rate: 1.0000e-04
Epoch 3/45
18/18 - 87s - 5s/step - accuracy: 0.8548 - loss: 4.5224 - val_accuracy: 0.2429 - val_loss: 6.6783 - learning_rate: 1.0000e-04
Epoch 4/45
18/18 - 85s - 5s/step - accuracy: 0.8639 - loss: 4.3890 - val_accuracy: 0.2599 - val_loss: 6.1718 - learning_rate: 1.0000e-04
Epoch 5/45
18/18 - 87s - 5s/step - accuracy: 0.8838 - loss: 4.2470 - val_accuracy: 0.2825 - val_loss: 6.0772 - learning_rate: 1.0000e-04
Epoch 6/45
18/18 - 85s - 5s/step - accuracy: 0.9238 - loss: 4.1184 - val_accuracy: 0.2542 - val_loss: 7.0858 - learning_rate: 1.0000e-04
Epoch 7/45
18/18 - 85s - 5s/step - accuracy: 0.9274 - loss: 3.9922 - val_accuracy: 0.2938 - val_loss: 6.5502 - learning_rate: 1.0000e-04
Epoch 8/45
18/18 - 85s - 5s/step - accur

0

In [8]:
print("\n\n==============================")
print("     TABLEAU FINAL DES SCORES")
print("==============================\n")

for r in results:
    print(f"{r['model']}: Accuracy = {r['accuracy']:.4f} | AUC = {r['auc']:.4f}")




     TABLEAU FINAL DES SCORES

ResNet50: Accuracy = 0.9322 | AUC = 0.9952
EfficientNetB0: Accuracy = 0.9605 | AUC = 0.9960
MobileNetV2: Accuracy = 0.8249 | AUC = 0.9693
